In [ ]:
from taming.modules.vqvae.quantize import VectorQuantizer2

In [ ]:
import torchvision.transforms.functional as TF
import torchvision.transforms as T
import torch
import PIL
from PIL import Image

def preprocess(img, target_image_size=256):
    s = min(img.size)
    
    if s < target_image_size:
        raise ValueError(f'min dim for image {s} < {target_image_size}')
        
    r = target_image_size / s
    s = (round(r * img.size[1]), round(r * img.size[0]))
    img = TF.resize(img, s, interpolation=PIL.Image.LANCZOS)
    img = TF.center_crop(img, output_size=2 * [target_image_size])
    img = torch.unsqueeze(T.ToTensor()(img), 0)
    return img

In [ ]:
import sys
print(sys.path)

In [ ]:
sys.path.append("./")

In [ ]:
from taming.modules.diffusionmodules.model import Encoder

In [ ]:
model = torch.load("/mnt/g/github/ZSE-SBIR/taming-transformers/logs/vqgan_imagenet_f16_1024/checkpoints/last.ckpt",map_location=torch.device('cuda:0'))

In [ ]:
from omegaconf import OmegaConf
config_path = "/mnt/g/github/ZSE-SBIR/taming-transformers/logs/vqgan_imagenet_f16_1024/configs/model.yaml"
config = OmegaConf.load(config_path)
print(config.model.params.ddconfig)

In [ ]:
enc_params = {'.'.join(key.split('.')[1:]): value for key,value in model['state_dict'].items() if key.startswith('encoder.')}
print(enc_params)

In [ ]:
enc = Encoder(**config.model.params.ddconfig)

enc.load_state_dict(enc_params)

In [ ]:
import cv2
import einops

In [ ]:
# im = cv2.imread("data/ade20k_segmentations/ADE_val_00000532.png")
# im = cv2.cvtColor(im,cv2.COLOR_BGR2RGB)

im = PIL.Image.open("/mnt/g/github/ZSE-SBIR/taming-transformers/data/coco_annotations_100/val2017/000000010092.jpg")
print(im.size)

im = preprocess(im)
# im = einops.rearrange(im.unsqueeze(0), 'b h w c -> b c h w')
print(im.shape)

In [ ]:
res = enc(im)

In [ ]:
quantizer = VectorQuantizer2(config.model.params.n_embed, config.model.params.embed_dim, 0.25)

In [ ]:
# for k,v in model['state_dict'].items():
#     if k.startswith('quantize'):
#         print(k)
quantizer_params =  {'.'.join(k.split('.')[1:]):v for k,v in model['state_dict'].items() if k.startswith('quantize')}
print(quantizer_params)
quantizer.load_state_dict(quantizer_params)

In [ ]:
res = quantizer(res)


In [ ]:
print(res[0].shape)

In [70]:
depth_config = config = OmegaConf.load("/mnt/g/github/ZSE-SBIR/taming-transformers/logs/2020-11-20T12-54-32_drin_transformer/configs/2020-11-20T12-54-32-project.yaml")
print(depth_config)
print(depth_config.model.params.transformer_config)


{'model': {'base_learning_rate': 4.5e-06, 'target': 'taming.models.cond_transformer.Net2NetTransformer', 'params': {'cond_stage_key': 'depth', 'transformer_config': {'target': 'taming.modules.transformer.mingpt.GPT', 'params': {'vocab_size': 1024, 'block_size': 512, 'n_layer': 24, 'n_head': 16, 'n_embd': 1024}}, 'first_stage_config': {'target': 'taming.models.vqgan.VQModel', 'params': {'ckpt_path': 'logs/2020-09-23T17-56-33_imagenet_vqgan/checkpoints/last.ckpt', 'embed_dim': 256, 'n_embed': 1024, 'ddconfig': {'double_z': False, 'z_channels': 256, 'resolution': 256, 'in_channels': 3, 'out_ch': 3, 'ch': 128, 'ch_mult': [1, 1, 2, 2, 4], 'num_res_blocks': 2, 'attn_resolutions': [16], 'dropout': 0.0}, 'lossconfig': {'target': 'taming.modules.losses.DummyLoss'}}}, 'cond_stage_config': {'target': 'taming.models.vqgan.VQModel', 'params': {'ckpt_path': 'logs/2020-11-03T15-34-24_imagenetdepth_vqgan/checkpoints/last.ckpt', 'embed_dim': 256, 'n_embed': 1024, 'ddconfig': {'double_z': False, 'z_chan

In [ ]:
from taming.models.cond_transformer import Net2NetTransformer
transformer = Net2NetTransformer(depth_config.model.params)